In [1]:
import cobra
from cobra.flux_analysis import production_envelope, pfba, flux_variability_analysis, moma
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
from cobra.medium import minimal_medium
from cobra.sampling import sample
from optlang import gurobi_interface

import pandas as pd
# from plotnine import ggplot, geom_point, aes, stat_smooth, facet_wrap

In [7]:
# import model

# model_path = '../../models/json/iML1515.json'
model_path = '../../models/json/GM_iML1515.json'
# model_path = '../../models/json/iJO1366.json'
model = cobra.io.load_json_model(model_path)

model.solver = 'glpk'
# model.solver = 'gurobi'

In [183]:
# test atp generation

with model:

    model.objective = 'ATPM'
    
    medium = model.medium
    # medium.update({'EX_glc__D_e': 1.0, 'EX_o2_e': 20.0})
    medium.update({'EX_glc__D_e': 1.0, 'EX_phleth_e': 0.0, 'EX_o2_e': 20.0})
    model.medium = medium
    
    model.reactions.get_by_id('ATPM').upper_bound = 1000.0
    
    solution = model.optimize()
    # solution = pfba(model)
    # print(dir(solution))
    # print(solution.reduced_costs)

    print('atp yield: ', solution.objective_value);
    print(model.metabolites.atp_c.summary());

    # from 1 glucose: 
    # 2 atp from pgk (glycolysis)
    # 2 atp from sucoas (tca cycle)
    # 19.5 atp from atps4rpp (atp synthase within periplasm)
    # total 23.5 atp produced

atp yield:  23.499999999999865
atp_c
=====
Formula: C10H12N5O13P3

Producing Reactions
-------------------
Percent Flux Reaction                                         Definition
 82.98% 19.5 ATPS4rpp adp_c + 4.0 h_p + pi_c <=> atp_c + h2o_c + 3.0 h_c
  8.51%    2      PGK                  3pg_c + atp_c <=> 13dpg_c + adp_c
  8.51%    2   SUCOAS atp_c + coa_c + succ_c <=> adp_c + pi_c + succoa_c

Consuming Reactions
-------------------
Percent  Flux Reaction                           Definition
100.00% -23.5     ATPM atp_c + h2o_c --> adp_c + h_c + pi_c


In [75]:
# wild type yield

with model:
    medium = model.medium
    # medium.update({'EX_glc__D_e': 1.0, 'EX_o2_e': 20.0})
    medium.update({'EX_glc__D_e': 0.0, 'EX_phleth_e': 10.0, 'EX_o2_e': 20.0})
    # medium.update({'EX_pi_e': 1000.0, 'EX_co2_e': 0, 'EX_fe3_e': 1000.0, 'EX_h_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_fe2_e': 0, 'EX_glc__D_e': 10.0, 'EX_zn2_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_ca2_e': 1000.0, 'EX_ni2_e': 0, 'EX_cu2_e': 1000.0, 'EX_sel_e': 0, 'EX_cobalt2_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_mobd_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_k_e': 1000.0, 'EX_na1_e': 1000.0, 'EX_cl_e': 1000.0, 'EX_o2_e': 20.0, 'EX_tungs_e': 0, 'EX_slnt_e': 0})
    model.medium = medium

    # model.add_boundary(model.metabolites.get_by_id('accoa_c'), type = 'demand')
    
    # model.objective = 'CS'
    # model.objective = 'DM_accoa_c'
    model.objective = 'DM_phb_c'
    # model.objective = {model.reactions.BIOMASS_Ec_iML1515_core_75p37M: 1, model.reactions.DM_phb_c: 0.05}
    
    solution = model.optimize()
    # pfba_solution = pfba(model)
    
    print('biomass yield: ', solution.fluxes.get('BIOMASS_Ec_iML1515_core_75p37M'));
    print('accoa yield: ', solution.fluxes.get('DM_accoa_c'))
    print('phb yield: ', solution.fluxes.get('DM_phb_c'))
    
    print(model.metabolites.accoa_c.summary());
    print(model.metabolites.phb_c.summary());
    
    print(model.summary())
    
#     prod_env = production_envelope(
#     model, ["EX_o2_e"], objective='DM_phb_c', carbon_sources="EX_glc__D_e")
    
    # cobra.io.save_matlab_model(model, "../../models/mat/test.mat")
    

biomass yield:  0.0
accoa yield:  None
phb yield:  9.238736842105272
accoa_c
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent    Flux Reaction                                         Definition
 33.33%   6.159  3OXCOAT          coa_c + oxadpcoa_c --> accoa_c + succoa_c
 33.33%   6.159 OXDHCOAT       3oxdhscoa_c + coa_c --> 23dhacoa_c + accoa_c
 33.15%   6.125      PDH coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
  0.19% 0.03453      PFL                  coa_c + pyr_c --> accoa_c + for_c

Consuming Reactions
-------------------
Percent   Flux Reaction                      Definition
100.00% -18.48  ACACT1r 2.0 accoa_c <=> aacoa_c + coa_c
phb_c
=====
Formula: C4H6O2

Producing Reactions
-------------------
Percent  Flux Reaction                    Definition
100.00% 9.239    PHACr hbcoa__3R_c <=> coa_c + phb_c

Consuming Reactions
-------------------
Percent   Flux Reaction Definition
100.00% -9.239 DM_phb_c phb_c --> 
Objective
1.0 DM_phb_c = 9.238736842105

In [76]:
%matplotlib inline
prod_env.plot(
    kind='line', x='EX_o2_e', y='carbon_yield_maximum');

NameError: name 'prod_env' is not defined

In [8]:
# fva to test overexpression thresholds

with model:
    
    medium = model.medium
    # medium.update({'EX_glc__D_e': 1.0, 'EX_o2_e': 20.0})
    medium.update({'EX_glc__D_e': 0.0, 'EX_phleth_e': 6.159, 'EX_o2_e': 20.0})
    # medium.update({'EX_pi_e': 1000.0, 'EX_co2_e': 0, 'EX_fe3_e': 1000.0, 'EX_h_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_fe2_e': 0, 'EX_glc__D_e': 10.0, 'EX_zn2_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_ca2_e': 1000.0, 'EX_ni2_e': 0, 'EX_cu2_e': 1000.0, 'EX_sel_e': 0, 'EX_cobalt2_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_mobd_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_k_e': 1000.0, 'EX_na1_e': 1000.0, 'EX_cl_e': 1000.0, 'EX_o2_e': 20.0, 'EX_tungs_e': 0, 'EX_slnt_e': 0})
    model.medium = medium

    # model.add_boundary(model.metabolites.get_by_id('accoa_c'), type = 'demand')
    
    # model.objective = 'CS'
    # model.objective = 'DM_accoa_c'
    # model.objective = 'DM_phb_c'
    
    model.optimize()
    
    fva_rxns = [model.reactions.ACS, model.reactions.PNTK]
    print(flux_variability_analysis(model, reaction_list = fva_rxns, fraction_of_optimum = 0.95, loopless = True))
    
    # print(model.summary(fva = 0.95))
    # print(model.metabolites.accoa_c.summary(fva = 0.95))


       minimum   maximum
ACS   0.000000  1.194521
PNTK  0.000196  0.000206


In [105]:
# pta + ack knockout, acs overexpression

with model:
    
    medium = model.medium
    # medium.update({'EX_glc__D_e': 1.0, 'EX_o2_e': 20.0})
    # medium.update({'EX_glc__D_e': 0.5, 'EX_phleth_e': 0.375, 'EX_o2_e': 20.0})
    medium.update({'EX_glc__D_e': 10.0, 'EX_phleth_e': 10.0, 'EX_o2_e': 20.0})
    # medium.update({'EX_pi_e': 1000.0, 'EX_co2_e': 0, 'EX_fe3_e': 1000.0, 'EX_h_e': 1000.0, 'EX_mn2_e': 1000.0, 'EX_fe2_e': 0, 'EX_glc__D_e': 10.0, 'EX_zn2_e': 1000.0, 'EX_mg2_e': 1000.0, 'EX_ca2_e': 1000.0, 'EX_ni2_e': 0, 'EX_cu2_e': 1000.0, 'EX_sel_e': 0, 'EX_cobalt2_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_mobd_e': 1000.0, 'EX_so4_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_k_e': 1000.0, 'EX_na1_e': 1000.0, 'EX_cl_e': 1000.0, 'EX_o2_e': 20.0, 'EX_tungs_e': 0, 'EX_slnt_e': 0})
    model.medium = medium
    
    
    model.add_boundary(model.metabolites.get_by_id('accoa_c'), type = 'demand')
    
    # model.objective = 'CS'
    # model.objective = 'DM_accoa_c'
    # model.objective = 'DM_phb_c'
    model.objective = {model.reactions.BIOMASS_Ec_iML1515_core_75p37M: 1, model.reactions.DM_phb_c: 0.02}

    solution = model.optimize()
    
    print('biomass yield wt: ', solution.fluxes.get('BIOMASS_Ec_iML1515_core_75p37M'))
    print('phb yield wt: ', solution.fluxes.get('DM_phb_c'))
    
    old = float(solution.fluxes.get('DM_phb_c'))
    
    # maintain biomass production
    # model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.1
    
    # acetate modifications
    # model.reactions.PTAr.knock_out() # pta
    # model.reactions.ACKr.knock_out() # ackA
    # model.reactions.ACS.lower_bound = 1.5 # acs
    
    # tca modifications
    model.reactions.SUCOAS.knock_out() # sucC (succinyl coa synthetase)
    model.reactions.SUCDi.knock_out() # sdhCAB (succinate dehydrogenase)
    # model.reactions.FUM.knock_out() # fumC (fumarase)

    # others
    # model.reactions.ACOAD6f.knock_out()
    # model.reactions.POX.knock_out() # POX (pyruvate oxidase)
    
    # minimum metabolite yield
    # model.reactions.DM_accoa_c.lower_bound = 0.1
    # model.reactions.DM_phb_c.lower_bound = 0.5
    
    # solution = model.optimize()
    # moma_solution = moma(model, solution)
    # print(dir(moma_solution))
    
    # print(model.summary(fva = 0.95))

    # cobra.io.save_matlab_model(model, "../../models/mat/test.mat")
    # cobra.io.save_json_model(model, "../../models/json/test.json")
    
    solution = model.optimize()

    print('biomass yield: ', solution.fluxes.get('BIOMASS_Ec_iML1515_core_75p37M'))
    # print('biomass yield (moma): ', moma_solution.fluxes.get('BIOMASS_Ec_iML1515_core_75p37M'))
    # print('ATP maintenance: ', solution.fluxes.get('ATPM'))
    # print('accoa yield: ', solution.fluxes.get('DM_accoa_c'))
    # print('accoa yield (moma): ', moma_solution.fluxes.get('DM_accoa_c'))
    print('phb yield: ', solution.fluxes.get('DM_phb_c'))
    # print('phb yield (moma): ', moma_solution.fluxes.get('DM_phb_c'))
    
    new = float(solution.fluxes.get('DM_phb_c'))
    try:
        dif = new-old
        pdif = (new-old)/old*100
        print('\nnumerical increase phb: ' + str(dif))
        print('percent increase phb: ' + str(pdif) +'%\n')
    except:
        print('\nfuck\n')
        pass
    
    print(model.metabolites.accoa_c.summary());
    # print(model.metabolites.nadh_c.summary());
    # print(model.metabolites.nadph_c.summary());
    # print(model.metabolites.cit_c.summary());
    # print(model.metabolites.acon_C_c.summary());
    # print(model.metabolites.icit_c.summary());
    # print(model.metabolites.akg_c.summary());
    # print(model.metabolites.succoa_c.summary());
    # print(model.metabolites.succ_c.summary());
    # print(model.metabolites.fum_c.summary());
    # print(model.metabolites.mal__L_c.summary());
    # print(model.metabolites.oaa_c.summary());
    # print(model.metabolites.acald_c.summary());
    # print(model.metabolites.acser_c.summary());
    # print(model.metabolites.cys__L_c.summary());
    # print(model.metabolites.get_by_id('4ppcys_c').summary());
    # print(model.metabolites.pan4p_c.summary());
    # print(model.metabolites.dpcoa_c.summary());
    # print(model.metabolites.coa_c.summary());
    # print(model.metabolites.ac_c.summary());
    # print(model.metabolites.phb_c.summary());
    # print(model.metabolites.phleth_c.summary());
    # print(model.metabolites.atp_c.summary());
    
    print(model.summary())
    # print(model.summary(fva = 0.95))
    
    # shadow = solution.shadow_prices
    # shadow.to_csv('shadow.csv')
    
    # t = sample(model, 20, processes = 2)


biomass yield wt:  0.7986056852720439
phb yield wt:  1.2255790293368727
biomass yield:  0.7451470719176659
phb yield:  1.738883728097967

numerical increase phb: 0.5133046987610943
percent increase phb: 41.882627433567414%

accoa_c
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent      Flux Reaction                                         Definition
  0.00% 0.0003338    PAI2T              coa_c + dhptdd_c --> accoa_c + dhap_c
 93.93%      6.72      PDH coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
  6.06%    0.4338     PTAr                  accoa_c + pi_c <=> actp_c + coa_c

Consuming Reactions
-------------------
Percent       Flux Reaction                                                 Definition
 56.03%     -4.008  ACACT1r                            2.0 accoa_c <=> aacoa_c + coa_c
  3.71%    -0.2652  ACACT2r                     accoa_c + btcoa_c <=> 3ohcoa_c + coa_c
  3.71%    -0.2652  ACACT3r                     accoa_c + hxcoa_c <=> 3oocoa_c + coa_c
  

In [270]:
s.to_csv('sampling/s.csv', index = False)
t.to_csv('sampling/t.csv', index = False)